In [4]:
from PIL import Image
import pytesseract
import fitz
import os

pdf_files = os.listdir('./pdfs')


In [ ]:
doc = fitz.open(pdf_file)
pages = doc.page_count

for page_num in range(pages):
    page = doc.load_page(page_num)
    image = page.get_pixmap()
    image.save(f'./pages/page_{page_num + 1}.png', 'png')
    image_path = f'./pages/page_{page_num + 1}.png'
    text = pytesseract.image_to_string(Image.open(image_path))
    print(text)
    
image_path = 'page_1.png'  # Adjust the path to the image you want to process
text = pytesseract.image_to_string(Image.open(image_path))

print(text)

14343
